# ECSE 415 - Assignment 5: Video Analysis

## Authors

| *Name*                         |*Student ID*|
|--------------------------------|------------|
| Ana Gordon                     | 261113440 |
| Mathias Nahuel Pacheco Lemina  | 261116679  |

## Introduction
The goal of this assignment is 

https://www.freecodecamp.org/news/how-to-detect-objects-in-images-using-yolov8/#heading-how-to-get-started-with-yolov8

https://www.geeksforgeeks.org/python/saving-a-video-using-opencv/ --> for 1 (data preparation)

https://www.kaggle.com/code/nityampareek/using-deepsort-object-tracker-with-yolov5#DeepSORT --> for 2.1

## `0` Preparation

Run this in a terminal first

*python3 -m venv venv && source venv/bin/activate && python -m pip install --upgrade pip && python -m pip install -r requirements.txt*

In [1]:
!pip install ultralytics
!pip install opencv-python
!pip install deep-sort-realtime
!pip install datetime
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Looking in indexes: https://download.pytorch.org/whl/cu121


### `Imports`

In [7]:
# Set environment vars BEFORE importing torch/CUDA initialization
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [18]:
# imports

import numpy as np
import cv2
import time
import subprocess
import shutil
from pathlib import Path
import sys
import glob
import torchvision
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import datetime
import os
import pandas as pd



### `Paths`

In [9]:
data_path = './Object_Tracking/'

task1_images_path = os.path.join(data_path, 'Task1/Images/')

## `1` Data Preparation

In [33]:
#1.1 convert images to video and 1.2 save output video
input_video = 'task1_input.mp4'

def get_video_writer(frame_size, filename):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(filename, fourcc, 14, frame_size)
    return writer

def create_input_video(source, filename):
    if isinstance(source, str) and os.path.isdir(source):
        images = sorted([p for p in os.listdir(source) if p.endswith('.jpg')])
        first = cv2.imread(os.path.join(source, images[0]))
        h, w = first.shape[:2]
        writer = get_video_writer((w, h), filename)
        for img_name in images:
            img = cv2.imread(os.path.join(source, img_name))
            writer.write(img)
        writer.release()
        return filename
        
create_input_video(task1_images_path, input_video)
print(f"Video saved to: {input_video}")


Video saved to: task1_input.mp4


## `2` Model Implementation

In [34]:
input_video = globals().get('input_video', 'task1_input.mp4')
output_video = 'task1.mp4'
output_txt = 'task1.txt'

# load model
model = YOLO('yolov8n.pt')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(f"Using device: {device}")

# CPU embedder for compatibility 
tracker = DeepSort(max_age=30, embedder_gpu=False) # 2.1 apply deepsort tracker

cap = cv2.VideoCapture(input_video)
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = get_video_writer((frame_w, frame_h), output_video)

# tracking results
f = open(output_txt, 'w')

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_number += 1

    results = model(frame, device=device)
    frame_result = results[0]  

    detections = []

    if hasattr(frame_result, 'boxes') and hasattr(frame_result.boxes, 'data') and frame_result.boxes.data is not None:
        boxes = frame_result.boxes.data.cpu().numpy() if hasattr(frame_result.boxes.data, 'cpu') else frame_result.boxes.data.numpy()
       
        for b in boxes:
            # bbox: [x1,y1,x2,y2,conf,class]
            x1, y1, x2, y2, conf, class_found = (b.tolist() if hasattr(b, 'tolist') else list(b))
            class_found = int(class_found)
            
            # keep only pedestrians (COCO class 0)
            if class_found  != 0:
                continue

            xmin = int(max(0, round(x1)))
            ymin = int(max(0, round(y1)))
            width = int(round(x2 - x1))
            height = int(round(y2 - y1))
            detections.append([[xmin, ymin, width, height], float(conf), class_found])

    # update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # 2.2 - draw bounding box and ID
    for tr in tracks:
        if not tr.is_confirmed():
            track_id = -1
            continue
        track_id = tr.track_id
        left, top, right, bottom = tr.to_ltrb()
        left, top, right, bottom  = int(left), int(top), int(right), int(bottom)
        width = right-left
        height = bottom-top
        
        # draw bbox and id
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f'ID: {track_id}'
        cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # results.txt: <frame>, <id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>
        f.write(f'{frame_number},{track_id},{left},{top},{width},{height}\n') # 2.4 writes to file task1.txt

    writer.write(frame)

# cleanup
f.close()
cap.release()
writer.release() # 2.3 saves output video as task1.mp4
cv2.destroyAllWindows()

Using device: cpu

0: 384x640 16 persons, 1 umbrella, 21.6ms
0: 384x640 16 persons, 1 umbrella, 21.6ms
Speed: 1.3ms preprocess, 21.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Speed: 1.3ms preprocess, 21.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


0: 384x640 18 persons, 1 umbrella, 18.6ms
Speed: 1.4ms preprocess, 18.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 18 persons, 1 umbrella, 18.6ms
Speed: 1.4ms preprocess, 18.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 1 umbrella, 16.8ms

0: 384x640 15 persons, 1 umbrella, 16.8ms
Speed: 1.1ms preprocess, 16.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Speed: 1.1ms preprocess, 16.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)


0: 384x640 15 persons, 1 umbrella, 18.5ms
Speed: 1.4ms preprocess, 18.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
0:

KeyboardInterrupt: 

## `3` Model Evaluation

In [35]:
# code in this section for model evaluation from Tutorial 12

# IoU >= 0.5
def iou(boxA, boxB):
    xA = max(boxA['x'], boxB['x'])
    yA = max(boxA['y'], boxB['y'])
    xB = min(boxA['x'] + boxA['w'], boxB['x'] + boxB['w'])
    yB = min(boxA['y'] + boxA['h'], boxB['y'] + boxB['h'])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA['w'] * boxA['h']
    boxBArea = boxB['w'] * boxB['h']
    if (boxAArea + boxBArea - interArea) <= 0:
        return 0
    return interArea / float(boxAArea + boxBArea - interArea)

# Hungarian Algorithm or equivalent (aka Greedy matching)
def greedy_match(gt_frame, pred_frame, iou_thresh=0.5):
    matches = []
    if len(gt_frame) == 0 or len(pred_frame) == 0:
        return matches

    # Compute IoU for all GT-Pred pairs on the same frame and only keep IoUs ≥ threshold:
    iou_matrix = []
    for i, gt_row in gt_frame.iterrows():
        for j, pred_row in pred_frame.iterrows():
            score = iou(gt_row, pred_row)
            if score >= iou_thresh:
                iou_matrix.append((score, i, j))

    # Sort descending by IoU
    iou_matrix.sort(key=lambda x: x[0], reverse=True)

    # greedy: You take the highest-IoU match first and assign it.
    # Then you block (mark used) the GT and prediction so no one else can use them.

    gt_used, pred_used = set(), set()

    for score, i, j in iou_matrix:
        if i not in gt_used and j not in pred_used:
            matches.append((i, j, score))
            gt_used.add(i)
            pred_used.add(j)
    return matches

# track and count
def compute_tracking_metrics(gt_file, pred_file, iou_thresh=0.5):
    gt_df = pd.read_csv(
        gt_file,
        header=None,
        usecols=[0,1,2,3,4,5],
        names=['frame','id','x','y','w','h']
    )

    pred_df = pd.read_csv(
        pred_file,
        header=None,
        usecols=[0,1,2,3,4,5],
        names=['frame','id','x','y','w','h']
    )

    frames = sorted(gt_df['frame'].unique())

    TP_total, FP_total, FN_total, IDSW_total = 0, 0, 0, 0
    prev_assignment = {}      # GT id → Pred id from previous frame

    for f in frames:
        gt_frame = gt_df[gt_df['frame'] == f].reset_index(drop=True)
        pred_frame = pred_df[pred_df['frame'] == f].reset_index(drop=True)

        matches = greedy_match(gt_frame, pred_frame, iou_thresh)

        # Count TP/FP/FN
        TP_total += len(matches)
        FP_total += len(pred_frame) - len(matches) # unmatched predictions
        FN_total += len(gt_frame) - len(matches)   # unmatched GT

        current_assignment = {}

        for i, j, _ in matches:
            gt_id = gt_frame.iloc[i]['id']
            pred_id = pred_frame.iloc[j]['id']
            current_assignment[gt_id] = pred_id

            # Identity switch
            # A GT id is tracked across frames.
            # If the matched predicted ID changes → identity switch.

            # Example:
            # Frame 10: GT 5 → Pred 12
            # Frame 11: GT 5 → Pred 19 ← identity switched

            if gt_id in prev_assignment and prev_assignment[gt_id] != pred_id:
                IDSW_total += 1


        # Update previous assignment for next frame
        prev_assignment = current_assignment

    # Precision / Recall / F1
    precision = TP_total / (TP_total + FP_total) if TP_total + FP_total > 0 else 0
    recall = TP_total / (TP_total + FN_total) if TP_total + FN_total > 0 else 0
    f1 = (2 * precision * recall / (precision + recall)
          if precision + recall > 0 else 0)

    # MOTA
    GT_total = len(gt_df)  # total GT boxes
    MOTA = 1 - (FN_total + FP_total + IDSW_total) / GT_total if GT_total > 0 else 0

    # Print metrics
    print("----- Tracking Metrics -----")
    print(f"Frames evaluated: {len(frames)}")
    print(f"True Positives (TP): {TP_total}")
    print(f"Ground Truth boxes (GT): {GT_total}")
    print(f"False Positives (FP): {FP_total}")
    print(f"False Negatives (FN): {FN_total}")
    print(f"Identity Switches (IDSW): {IDSW_total}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"MOTA: {MOTA:.4f}")

# print metrics
compute_tracking_metrics(
    gt_file=os.path.join(data_path, 'Task1/gt/gt.txt'),
    pred_file=output_txt,
    iou_thresh=0.5
)

----- Tracking Metrics -----
Frames evaluated: 429
True Positives (TP): 6099
Ground Truth boxes (GT): 26647
False Positives (FP): 2003
False Negatives (FN): 20548
Identity Switches (IDSW): 158
Precision: 0.7528
Recall: 0.2289
F1-score: 0.3510
MOTA: 0.1478


## `4` Prediction & Kaggle Competition